<a href="https://colab.research.google.com/github/EddieAtGoogle/notebooks/blob/main/Retrieval_Aug_Generation_BQML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall shapely -y
!pip install google-cloud-aiplatform

# **Authenticate with your GCP credentials**

In [ ]:
from google.colab import auth
auth.authenticate_user()

## **Optional: Enable data table display**

Colab includes the ``google.colab.data_table`` package that can be used to display large pandas dataframes as an interactive data table.
It can be enabled with:

In [ ]:
%load_ext google.colab.data_table

# **Use BigQuery via magics**

The `google.cloud.bigquery` library also includes a magic command which runs a query and either displays the result or saves it to a variable as a `DataFrame`.

In [ ]:
%%bigquery --project tinsleydataengineering
SELECT "Review 1" AS title, "The dishwasher was amazing.  My dishes have never been so clean" AS content
UNION ALL
SELECT "Review 2" AS title, "My dishes came out of this thing more dirty than when I put them in.  Refund please!" AS content
UNION ALL
SELECT "Review 3" AS title, "I loved the stainless steel finish and the modern aesthetic but the service leaves a lot to be desired." AS content
UNION ALL
SELECT "Review 4" AS title, "Meh." AS content

# **Create remote GenAI models**

Create remote models based on the Vertex AI text-bison and textembedding-gecko foundation models that will allow us to perform several text generation & embeding tasks via BQML

In [ ]:
# Create remote model for text generation

%%bigquery --project tinsleydataengineering
CREATE OR REPLACE MODEL
`tinsleydataengineering.bqml_genai_models.bqml_text_bison`
REMOTE WITH CONNECTION `tinsleydataengineering.us.tde_vertex_bqml_remote_connection`
OPTIONS (REMOTE_SERVICE_TYPE = 'CLOUD_AI_LARGE_LANGUAGE_MODEL_V1');

In [ ]:
# Create remote model for text embeddings

%%bigquery --project tinsleydataengineering
CREATE OR REPLACE MODEL
`tinsleydataengineering.bqml_genai_models.bqml_text_embedding`
REMOTE WITH CONNECTION `tinsleydataengineering.us.tde_vertex_bqml_remote_connection`
OPTIONS (REMOTE_SERVICE_TYPE = 'CLOUD_AI_TEXT_EMBEDDING_MODEL_V1');

In [ ]:
# Use text bison to generate text

%%bigquery --project tinsleydataengineering
SELECT *
FROM ML.GENERATE_TEXT(
  MODEL `tinsleydataengineering.bqml_genai_models.bqml_text_bison`,
  (SELECT "How does a dishwasher work?" as prompt),
  STRUCT(1024 AS max_output_tokens, 1.0 AS temperature,
  40 AS top_k, 0.8 AS top_p, TRUE AS flatten_json_output)
);

Query is running:   0%|          |

Downloading:   0%|          |

,ml_generate_text_llm_result,ml_generate_text_rai_result,ml_generate_text_status,prompt
0,A dishwasher is a machine that cleans dishes ...,"{""blocked"":false,""categories"":[],""scores"":[]}",,How does a dishwasher work?


In [ ]:
# Use BQML to generate text embeddings

%%bigquery --project tinsleydataengineering
SELECT *
FROM ML.GENERATE_TEXT_EMBEDDING(
  MODEL `tinsleydataengineering.bqml_genai_models.bqml_text_embedding`,
  (SELECT "Review 1" AS title, "The dishwasher was amazing.  My dishes have never been so clean" AS content
        UNION ALL
        SELECT "Review 2" AS title, "My dishes came out of this thing more dirty than when I put them in.  Refund please!" AS content
        UNION ALL
        SELECT "Review 3" AS title, "I loved the stainless steel finish and the modern aesthetic but the service leaves a lot to be desired." AS content
        UNION ALL
        SELECT "Review 4" AS title, "Meh." AS content),
  STRUCT(TRUE AS flatten_json_output)
);

In [ ]:
# Save output in a variable `df`

%%bigquery df --project tinsleydataengineering
SELECT
  COUNT(*) as total_rows
FROM `tinsleydataengineering.appliances_mock_data.consumer_review`

In [ ]:
df

# Use BigQuery through google-cloud-bigquery python SDK to generate mock reviews using text bison

See [BigQuery documentation](https://cloud.google.com/bigquery/docs) and [library reference documentation](https://googlecloudplatform.github.io/google-cloud-python/latest/bigquery/usage.html).


### Declare the Cloud project ID which will be used by the Python SDK and instantiate the BigQuery client

In [ ]:
project_id = 'tinsleydataengineering'

In [ ]:
from google.cloud import bigquery

client = bigquery.Client(project=project_id)

# **Load review metadata into a dataframe.  We'll use this metadata to generate the prompts we'll used to generate mock reviews**

In [ ]:
review_metadata = client.query('''
SELECT brand,
       product_line,
       generated_sentiment,
       "" AS content,
       review_ts
FROM `tinsleydataengineering.appliances_mock_data.consumer_review`
''').to_dataframe()

review_metadata.head(5)

,brand,product_line,generated_sentiment,content,review_ts
0,LGE,Dryer,angry,,2022-04-28 02:14:02+00:00
1,Kosch,Dryer,angry,,2023-02-25 20:07:10+00:00
2,LGE,Dryer,angry,,2022-03-03 20:56:17+00:00
3,Hisung,Dryer,angry,,2022-07-04 19:33:18+00:00
4,LGE,Dryer,angry,,2023-03-10 11:10:43+00:00


# **Create a new data frame column named prompt and populate it with a review generation LLM prompt**

In [ ]:
review_metadata['prompt'] = review_metadata.apply(lambda x: 'Generate a creative and unique consumer review of a recently purchased appliance with the following attributes.\nBrand: {}\nAppliance: {}\nSentiment: {}'.format(x['brand'], x['product_line'], x['generated_sentiment']), axis=1)
review_metadata.head(5)

,brand,product_line,generated_sentiment,content,review_ts,prompt
0,LGE,Dryer,angry,,2022-04-28 02:14:02+00:00,Generate a creative and unique consumer review...
1,Kosch,Dryer,angry,,2023-02-25 20:07:10+00:00,Generate a creative and unique consumer review...
2,LGE,Dryer,angry,,2022-03-03 20:56:17+00:00,Generate a creative and unique consumer review...
3,Hisung,Dryer,angry,,2022-07-04 19:33:18+00:00,Generate a creative and unique consumer review...
4,LGE,Dryer,angry,,2023-03-10 11:10:43+00:00,Generate a creative and unique consumer review...


# **Create a BigQuery load job to load the newly created prompts**


In [ ]:
table_id = "tinsleydataengineering.appliances_mock_data.consumer_review_working"

job_config = bigquery.LoadJobConfig(
    # Specify a (partial) schema. All columns are always written to the
    # table. The schema is used to assist in data type definitions.
    schema=[
        # Specify the type of columns whose type cannot be auto-detected. For
        # example the "title" column uses pandas dtype "object", so its
        # data type is ambiguous.
        bigquery.SchemaField("content", bigquery.enums.SqlTypeNames.STRING),
    ],
    # Optionally, set the write disposition. BigQuery appends loaded rows
    # to an existing table by default, but with WRITE_TRUNCATE write
    # disposition it replaces the table with the loaded data.
    write_disposition="WRITE_TRUNCATE",
)

job = client.load_table_from_dataframe(
    review_metadata, table_id, job_config=job_config
)  # Make an API request.
job.result()  # Wait for the job to complete.

table = client.get_table(table_id)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)

# **Use text bison to generate synthetic consumer reviews via BQML**

In [ ]:
%%bigquery --project tinsleydataengineering
INSERT INTO tinsleydataengineering.appliances_mock_data.consumer_review_working (brand, product_line, generated_sentiment, content, review_ts, prompt)
SELECT brand, product_line, generated_sentiment, ml_generate_text_llm_result AS content, review_ts, prompt
FROM
  ML.GENERATE_TEXT(
    MODEL tinsleydataengineering.bqml_genai_models.bqml_text_bison,
    TABLE tinsleydataengineering.appliances_mock_data.consumer_review_working,
    STRUCT(
      1.0 AS temperature, 512 AS max_output_tokens, 0.8 AS top_p,
      40 AS top_k, TRUE AS flatten_json_output));

# **Add column to working table to store text embeddings**

In [ ]:
%%bigquery --project tinsleydataengineering
ALTER TABLE tinsleydataengineering.appliances_mock_data.consumer_review_working
  ADD COLUMN IF NOT EXISTS text_embedding ARRAY<FLOAT64> OPTIONS(description="Text embedding representation of data in the content column")

# **Use BQML to generate text embeddings**

In [ ]:
# Test query to validate function of the text embedding model

%%bigquery --project tinsleydataengineering
SELECT brand, product_line, generated_sentiment, content, text_embedding, review_ts, prompt
FROM ML.GENERATE_TEXT_EMBEDDING(
  MODEL `tinsleydataengineering.bqml_genai_models.bqml_text_embedding`,
  (SELECT brand, product_line, generated_sentiment, content, review_ts, prompt FROM tinsleydataengineering.appliances_mock_data.consumer_review_working WHERE content != "" LIMIT 5),
    STRUCT(TRUE AS flatten_json_output));

In [ ]:
# Generate embeddings and load them into the text_embedding column of the BigQuery table

%%bigquery --project tinsleydataengineering
INSERT INTO tinsleydataengineering.appliances_mock_data.consumer_review_working (brand, product_line, generated_sentiment, content, text_embedding, review_ts, prompt)
SELECT brand, product_line, generated_sentiment, content, text_embedding, review_ts, prompt
FROM ML.GENERATE_TEXT_EMBEDDING(
  MODEL `tinsleydataengineering.bqml_genai_models.bqml_text_embedding`,
  (SELECT brand, product_line, generated_sentiment, content, review_ts, prompt FROM tinsleydataengineering.appliances_mock_data.consumer_review_working WHERE content != ""),
    STRUCT(TRUE AS flatten_json_output));

In [ ]:
# Test query to validate embedding loaded into BigQuery

%%bigquery --project tinsleydataengineering
SELECT brand, product_line, generated_sentiment, content, text_embedding, review_ts, prompt
FROM tinsleydataengineering.appliances_mock_data.consumer_review_working WHERE ARRAY_LENGTH(text_embedding) != 0
LIMIT 5

# **Use BQML to perform a similarity search via the ML.DISTANCE function**

In [ ]:
%%bigquery --project tinsleydataengineering
WITH review_context_search AS (
  SELECT text_embedding
  FROM
    ML.GENERATE_TEXT_EMBEDDING(
      MODEL `tinsleydataengineering.bqml_genai_models.bqml_text_embedding`,
      (SELECT "How do LGE and Kosch dryer capacities compare?" AS content),
      STRUCT(TRUE AS flatten_json_output)
    )
)
SELECT
  content,
  ML.DISTANCE(
    (SELECT text_embedding FROM review_context_search),
    text_embedding,
    'COSINE'
  ) AS distance_to_average_review
FROM
  tinsleydataengineering.appliances_mock_data.consumer_review_working vector_store
WHERE ARRAY_LENGTH(vector_store.text_embedding) != 0
ORDER BY distance_to_average_review
LIMIT 5;

Query is running:   0%|          |

Downloading:   0%|          |

,content,distance_to_average_review
0,"After my old dryer finally gave out, I decide...",0.262547
1,"After 10 years of use, our old dryer finally ...",0.265152
2,"After my old dryer finally gave out, I spent ...",0.269100
3,I was in the market for a new dryer and decid...,0.270594
4,"After my old dryer finally gave out, I spent ...",0.270813


# Use the Vertex AI Python SDK to leverage the text embedding and generation models to implement retrieval augmented generation to answer user queries about the mock consumer reviews

In [ ]:
import vertexai
from vertexai.language_models import TextGenerationModel

vertexai.init(project="tinsleydataengineering", location="us-central1")
parameters = {
    "max_output_tokens": 512,
    "temperature": 0.8,
    "top_p": 0.8,
    "top_k": 40
}
model = TextGenerationModel.from_pretrained("text-bison")

In [ ]:
prompt = "\'What upsets customers about LGE dishwashers?\'"

context_query = f"""WITH review_context_search AS (
  SELECT text_embedding
  FROM
    ML.GENERATE_TEXT_EMBEDDING(
      MODEL `tinsleydataengineering.bqml_genai_models.bqml_text_embedding`,
      (SELECT {prompt} AS content),
      STRUCT(TRUE AS flatten_json_output)
    )
)
SELECT
  generated_sentiment,
  content,
  ML.DISTANCE(
    (SELECT text_embedding FROM review_context_search),
    text_embedding,
    'COSINE'
  ) AS distance_to_average_review
FROM
  tinsleydataengineering.appliances_mock_data.consumer_review_working vector_store
WHERE ARRAY_LENGTH(vector_store.text_embedding) != 0
ORDER BY distance_to_average_review
LIMIT 10;
"""

#print(context_query)

retrieved_llm_context = client.query(context_query).to_dataframe()

llm_context = ""

# Iterate through the reviews retrieved from BigQuery (may use a vector database in production)
for index, row in retrieved_llm_context.iterrows():
    review = row['content']
    sentiment = row['generated_sentiment']

    #print(sentiment + ": " + review)
    llm_context += sentiment + ": " + review + "\n\n"

print(llm_context)


In [ ]:
llm_prompt = f"""You are a helpful chat bot that answers questions based on customer reviews.
Please use the following customer review as context when answering the question.
The customer reviews are provided in the following format: (customer sentiment): (customer review).
Where possible reference the most relevant snippet from a customer review to support your response.

{llm_context}

Question: {prompt}
"""

print(llm_prompt)

In [ ]:
response = model.predict(llm_prompt, **parameters)

print(f"Response from Model: {response.text}")

Response from Model:  Customers are upset about LGE dishwashers because they do not clean the dishes properly and leave them with a white film. Many customers have tried using different detergents and rinse aids, but nothing seems to work. Some customers have also contacted LGE customer service, but they have been unable to resolve the issue.
